### 筛选数据

In [5]:
import pandas as pd
df = pd.read_csv('states_2020-04-20-00.csv')
# df.columns
# df.head(5)
# df.loc[df['onground']=='True']

# df.loc[df['onground']=='False']
data = pd.DataFrame(df,columns=[ 'icao24','time', 'lat', 'lon', 'geoaltitude'])
data.dropna(axis=0, inplace=True)

data.info()
data.head(10)
data.to_csv('data.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674047 entries, 0 to 674046
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   icao24       674047 non-null  object 
 1   time         674047 non-null  int64  
 2   lat          553709 non-null  float64
 3   lon          553709 non-null  float64
 4   geoaltitude  501143 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 25.7+ MB


In [13]:
spec_data = data[data['icao24'] == 'aa600a']
spec_data.head(20)



,icao24,time,lat,lon,geoaltitude
0,aa600a,1587340810,43.197171,-73.613303,11391.90
1693,aa600a,1587340820,43.199524,-73.635162,11391.90
3720,aa600a,1587340830,43.201175,-73.655875,11399.52
4980,aa600a,1587340840,43.202711,-73.677586,11391.90
7429,aa600a,1587340850,43.204376,-73.700377,11399.52
8454,aa600a,1587340860,43.205978,-73.721519,11399.52
10462,aa600a,1587340870,43.207646,-73.743373,11399.52
12543,aa600a,1587340880,43.209368,-73.766000,11399.52
14719,aa600a,1587340890,43.210968,-73.787246,11407.14
16476,aa600a,1587340900,43.212662,-73.810176,11407.14


### 经纬度转换

In [ ]:
import math
def trans_lon(lon):
    lon =  float(lon)*20037508.342789/180
    return lon
def trans_lat(lat):
    lat =math.log(math.tan((90+lat)*math.pi/360))/(math.pi/180)
    lat = lat *20037508.34789/180
    return lat

trans_lat(43.197171)

### 转换时间

In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')
# df.head(10)
# 经纬度转换
# df['lat2'] = df['lat'].map(trans_lat)
# df['lon2'] = df['lon'].map(trans_lon)

# 转换时间
df['date'] = pd.to_datetime(df['time'], unit='s')
df.head(20)

In [16]:
import pandas as pd
df = pd.read_csv('data.csv')
id_list = df['icao24'].unique()
len(id_list)
df.isnull().sum()
# df.isna().sum(axis=0)

icao24              0
time                0
lat            120338
lon            120338
geoaltitude    172904
dtype: int64

### 分组，选择数据

In [13]:
# data = df.groupby(by=[df['icao24']])
# for group_index, group in data:
#     print(group, group.shape)
grouped_data = df.groupby(by=[df['icao24']])
# for name,group in grouped_data:
#     for index in group.iteritems:
#         print(index)
# 选择数据
def select_data(data):
    data.sort_values(by='time')
    return data[::6]
filter_data = grouped_data.apply(select_data)
filter_data.isna().sum(axis=1)
# filter_data.to_csv('filter_data.csv', index=False)

icao24        
000001  571402    3
        583166    3
        593481    3
        605345    3
        615366    3
                 ..
e8045b  202397    0
        214019    0
        226384    0
        239565    0
        250092    0
Length: 113755, dtype: int64

In [24]:
# 根据时间筛选数据
import pandas as pd
data = pd.read_csv('filter_data.csv')
# data.head(20)
data.isna().sum(axis=0)

icao24         0
time           0
lat            0
lon            0
geoaltitude    0
date           0
dtype: int64

In [19]:
# 根据时间筛选数据
import pandas as pd
data = pd.read_csv('filter_data.csv')
# data.head(20)


def new_data_frame(origin, index):
    new_data = pd.DataFrame(columns=origin.columns)
    new_data['time'] = new_data['time'].astype('float64')
    return new_data.append(origin.iloc[index])
new_data_frame(data, 0)

process_index = 0
real_datas = []
id_list = data['icao24'].unique()
for i in range(0, len(id_list)):
    id = id_list[i]
    spec_data = data[data['icao24'] == id]
    if len(spec_data) == 0:
        continue
    spec_data.sort_values(by='time', inplace=True)

    # print("label is %s, size is %d" % (reg, len(spec_data)))
    real_data = new_data_frame(spec_data, 0)
    index = 1
    while index < len(spec_data):
        time_sep = spec_data.iloc[index]['time'] - spec_data.iloc[index - 1]['time']

        if time_sep < 60:
            pass
        elif time_sep == 60:
            real_data = real_data.append(spec_data.iloc[index])
        else:
            real_datas.append(real_data)
            real_data = new_data_frame(spec_data, index)

        index += 1

    real_datas.append(real_data)

    print(str(process_index) + ' size = ' + str(len(real_datas)))
    process_index += 1
# 导出数据
all_concat = pd.concat(real_datas, axis=0)
all_concat.to_csv('all_concat.csv',index=False)

D:\devsoft\conda\envs\aix\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0 size = 1
1 size = 2
2 size = 3
3 size = 4
4 size = 5
5 size = 6
6 size = 7
7 size = 8
8 size = 9
9 size = 10
10 size = 11
11 size = 12
12 size = 14
13 size = 15
14 size = 16
15 size = 19
16 size = 20
17 size = 21
18 size = 22
19 size = 23
20 size = 24
21 size = 25
22 size = 27
23 size = 28
24 size = 29
25 size = 30
26 size = 31
27 size = 32
28 size = 33
29 size = 34
30 size = 35
31 size = 36
32 size = 37
33 size = 38
34 size = 40
35 size = 41
36 size = 42
37 size = 43
38 size = 44
39 size = 45
40 size = 46
41 size = 47
42 size = 49
43 size = 51
44 size = 52
45 size = 54
46 size = 55
47 size = 57
48 size = 60
49 size = 61
50 size = 62
51 size = 63
52 size = 64
53 size = 65
54 size = 67
55 size = 68
56 size = 69
57 size = 70
58 size = 72
59 size = 74
60 size = 75
61 size = 76
62 size = 77
63 size = 79
64 size = 80
65 size = 81
66 size = 83
67 size = 85
68 size = 87
69 size = 88
70 size = 89
71 size = 90
72 size = 91
73 size = 94
74 size = 95
75 size = 96
76 size = 97
77 size = 99
78 si